In [57]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### Configuration

In [58]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm

DATA_DIR = "data/physionet.org/files/mimiciii/1.4/"
OUTPUT_FILE = "clean_dataset.csv"

# Vital item IDs (MIMIC-III standard)
VITALS = {
    "HR": [220045],
    "SysBP": [220179, 51],
    "DiasBP": [220180, 8368],
    "MeanBP": [220181],
    "RR": [220210],
    "SpO2": [220277],
    "TempC": [223761, 678],
}

# Lab item IDs
LABS = {
    "WBC": [51300],
    "HGB": [51222],
    "Platelets": [51265],
    "Sodium": [50983],
    "Potassium": [50971],
    "Creatinine": [50912],
    "BUN": [51006],
}


vital_ids = set([i for ids in VITALS.values() for i in ids])
lab_ids = set([i for ids in LABS.values() for i in ids])

### Loading Core Tables

In [ ]:
def load_core_tables():
    print("Loading core tables")

    patients = pd.read_csv(DATA_DIR + "PATIENTS.csv.gz", compression="gzip", parse_dates=["DOB"])

    admissions = pd.read_csv(DATA_DIR + "ADMISSIONS.csv.gz", compression="gzip", parse_dates=["ADMITTIME", "DISCHTIME"])

    icustays = pd.read_csv(DATA_DIR + "ICUSTAYS.csv.gz", compression="gzip", parse_dates=["INTIME", "OUTTIME"])

    diagnoses = pd.read_csv(DATA_DIR + "DIAGNOSES_ICD.csv.gz", compression="gzip")

    return patients, admissions, icustays, diagnoses

### Merging Core Tables on ID

In [ ]:
def merge_core(patients, admissions, icustays):
    print("Merging PATIENTS + ADMISSIONS + ICUSTAYS")

    # --------------------------------------------------------
    # 1. Clean DOB
    # --------------------------------------------------------
    patients["DOB"] = pd.to_datetime(patients["DOB"], errors="coerce")
    patients = patients[patients["DOB"].notna()]
    patients = patients[patients["DOB"].dt.year.between(1870, 2200)]

    # --------------------------------------------------------
    # 2. Merge PATIENTS + ADMISSIONS
    # --------------------------------------------------------
    adm_pat = admissions.merge(patients, on="SUBJECT_ID", how="left")

    adm_pat["ADMITTIME"] = pd.to_datetime(adm_pat["ADMITTIME"], errors="coerce")
    adm_pat["DISCHTIME"] = pd.to_datetime(adm_pat["DISCHTIME"], errors="coerce")

    adm_pat = adm_pat.dropna(subset=["ADMITTIME", "DISCHTIME"])
    adm_pat = adm_pat[adm_pat["ADMITTIME"].dt.year.between(2000, 3000)]
    adm_pat = adm_pat[adm_pat["DISCHTIME"].dt.year.between(2000, 3000)]

    # --------------------------------------------------------
    # 3. Merge ICU stays
    # --------------------------------------------------------
    merged = adm_pat.merge(
        icustays,
        on=["SUBJECT_ID", "HADM_ID"],
        how="left"
    )

    merged["INTIME"] = pd.to_datetime(merged["INTIME"], errors="coerce")
    merged["OUTTIME"] = pd.to_datetime(merged["OUTTIME"], errors="coerce")

    # --------------------------------------------------------
    # 4. AGE CALCULATION
    # --------------------------------------------------------

    y1 = merged["ADMITTIME"].dt.year
    m1 = merged["ADMITTIME"].dt.month
    d1 = merged["ADMITTIME"].dt.day

    y0 = merged["DOB"].dt.year
    m0 = merged["DOB"].dt.month
    d0 = merged["DOB"].dt.day

    age = y1 - y0

    birthday_not_reached = (m1 < m0) | ((m1 == m0) & (d1 < d0))
    age = age - birthday_not_reached.astype(int)

    valid_age = (
        merged["ADMITTIME"].notna() &
        merged["DOB"].notna() &
        y1.between(1900, 2300) &
        y0.between(1900, 2300) &
        (age >= 0) &
        (age < 200)
    )

    age = age.astype("float")
    age[~valid_age] = np.nan
    age[age > 120] = 90

    merged["AGE"] = age

    return merged

### Computing the 30-day Readmission Label

In [ ]:
def compute_readmission(df):
    print("Computing 30-day readmission labels")

    df = df.sort_values(["SUBJECT_ID", "ADMITTIME"])

    df["NEXT_ADMITTIME"] = df.groupby("SUBJECT_ID")["ADMITTIME"].shift(-1)
    df["HOURS_TO_NEXT"] = (
        (df["NEXT_ADMITTIME"] - df["DISCHTIME"]).dt.total_seconds() / 3600
    )

    df["READMIT_30D"] = (df["HOURS_TO_NEXT"] <= 720).astype(int)
    df["READMIT_30D"] = df["READMIT_30D"].fillna(0)

    return df

### Diagnosis-based Comorbidity Features

In [ ]:
def create_comorbidities(diagnoses):
    print("Generating diagnosis-based comorbidity features (Elixhauser)")

    # Load Elixhauser mappings (ICD9 prefix → binary comorbidity)
    # For simplicity, use common ICD9→Elixhauser mapping
    elix_map = {
        "250": "Diabetes",
        "428": "CHF",
        "414": "CAD",
        "401": "Hypertension",
        "585": "Renal_Failure",
        "491": "COPD",
        "571": "Liver_Disease",
        "518": "Respiratory_Failure",
        "410": "AMI",
        "038": "Sepsis",
    }

    diagnoses["icd_prefix"] = diagnoses["ICD9_CODE"].astype(str).str[:3]
    diagnoses["flag"] = 1

    pivot = {}
    for prefix, name in elix_map.items():
        filtered = diagnoses[diagnoses["icd_prefix"] == prefix]
        flags = filtered.pivot_table(
            index="HADM_ID",
            values="flag",
            aggfunc="max"
        ).rename(columns={"flag": name})
        pivot[name] = flags

    comorb = pd.concat(pivot.values(), axis=1)
    comorb = comorb.fillna(0)

    return comorb


### Vitals from the first 24 hours of ICU Admission

In [ ]:
def extract_vitals(merged):
    print("Extracting 24-hour vitals (efficient)")

    hadm_info = merged[["HADM_ID", "ADMITTIME"]].drop_duplicates()
    hadm_info = hadm_info.dropna(subset=["HADM_ID", "ADMITTIME"])
    hadm_info["HADM_ID"] = hadm_info["HADM_ID"].astype("int64")

    chunksize = 3_000_000
    rows = []

    for chunk in pd.read_csv(
        DATA_DIR + "CHARTEVENTS.csv.gz",
        compression="gzip",
        chunksize=chunksize,
        usecols=["HADM_ID", "ITEMID", "CHARTTIME", "VALUENUM"]
    ):
        chunk = chunk.dropna(subset=["HADM_ID"])
        chunk["HADM_ID"] = chunk["HADM_ID"].astype("int64")
        chunk = chunk[chunk["ITEMID"].isin(vital_ids)]
        chunk["CHARTTIME"] = pd.to_datetime(chunk["CHARTTIME"])

        # Merge admission times onto chunk
        merged_chunk = chunk.merge(hadm_info, on="HADM_ID", how="inner")

        # Keep rows within first 24 hours
        window_start = merged_chunk["ADMITTIME"]
        window_end = merged_chunk["ADMITTIME"] + pd.Timedelta(hours=24)

        mask = (merged_chunk["CHARTTIME"] >= window_start) & (
            merged_chunk["CHARTTIME"] <= window_end
        )

        merged_chunk = merged_chunk[mask]

        if merged_chunk.empty:
            continue

        # Aggregate: mean/min/max per HADM_ID per vital
        grp = merged_chunk.groupby(["HADM_ID", "ITEMID"])["VALUENUM"].agg(["mean", "min", "max"]).reset_index()
        rows.append(grp)

    if not rows:
        return pd.DataFrame(columns=["HADM_ID"])

    vitals_raw = pd.concat(rows)

    # Pivot to wide format
    vitals_wide = vitals_raw.pivot_table(
        index="HADM_ID",
        columns="ITEMID",
        values=["mean", "min", "max"]
    )

    vitals_wide.columns = [f"{stat}_{itemid}" for stat, itemid in vitals_wide.columns]
    vitals_wide = vitals_wide.reset_index()

    return vitals_wide

### Lab Results from the first 24 hours of ICU Admission

In [ ]:
def extract_labs(merged):
    print("Extracting 24-hour labs (efficient)")

    hadm_info = merged[["HADM_ID", "ADMITTIME"]].drop_duplicates()
    hadm_info = hadm_info.dropna(subset=["HADM_ID", "ADMITTIME"])
    hadm_info["HADM_ID"] = hadm_info["HADM_ID"].astype("int64")

    chunksize = 3_000_000
    rows = []

    for chunk in pd.read_csv(
        DATA_DIR + "LABEVENTS.csv.gz",
        compression="gzip",
        chunksize=chunksize,
        usecols=["HADM_ID", "ITEMID", "CHARTTIME", "VALUENUM"]
    ):
        # Drop missing HADM_ID
        chunk = chunk.dropna(subset=["HADM_ID"])
        chunk["HADM_ID"] = chunk["HADM_ID"].astype("int64")

        # Filter only labs we care about
        chunk = chunk[chunk["ITEMID"].isin(lab_ids)]

        # Parse timestamp
        chunk["CHARTTIME"] = pd.to_datetime(chunk["CHARTTIME"], errors="coerce")

        # Merge admission start times onto this chunk
        merged_chunk = chunk.merge(hadm_info, on="HADM_ID", how="inner")

        # Filter for first 24 hours
        starts = merged_chunk["ADMITTIME"]
        ends = starts + pd.Timedelta(hours=24)

        mask = (merged_chunk["CHARTTIME"] >= starts) & (merged_chunk["CHARTTIME"] <= ends)
        merged_chunk = merged_chunk[mask]

        if merged_chunk.empty:
            continue

        # Compute mean/min/max per lab item per admission
        grp = (
            merged_chunk
            .groupby(["HADM_ID", "ITEMID"])["VALUENUM"]
            .agg(["mean", "min", "max"])
            .reset_index()
        )

        rows.append(grp)

    if not rows:
        return pd.DataFrame(columns=["HADM_ID"])

    labs_raw = pd.concat(rows)

    # Pivot the ITEMID columns into wide feature columns
    labs_wide = labs_raw.pivot_table(
        index="HADM_ID",
        columns="ITEMID",
        values=["mean", "min", "max"]
    )

    labs_wide.columns = [f"{stat}_{itemid}" for stat, itemid in labs_wide.columns]
    labs_wide = labs_wide.reset_index()

    return labs_wide

### Final Merging

In [ ]:
def merge_all(merged, comorb, vital24, lab24):
    print("Merging all features into final dataset")

    df = (
        merged
        # Diagnosis features: by admission
        .merge(comorb, on="HADM_ID", how="left")

        # Vitals: now by admission
        .merge(vital24, on="HADM_ID", how="left")

        # Labs: also by admission
        .merge(lab24, on="HADM_ID", how="left")
    )

    # Final cleaning
    df = df.fillna(df.median(numeric_only=True))
    return df

### Full Dataset Preparation Pipeline

In [68]:

patients, admissions, icustays, diagnoses = load_core_tables()
merged = merge_core(patients, admissions, icustays)
merged = compute_readmission(merged)

comorb = create_comorbidities(diagnoses)
vital24 = extract_vitals(merged)


Loading core tables...
Merging PATIENTS + ADMISSIONS + ICUSTAYS...
Computing 30-day readmission labels...
Generating diagnosis-based comorbidity features (Elixhauser)...
Extracting 24-hour vitals (efficient)...


In [ ]:
lab24 = extract_labs(merged)

final = merge_all(merged, comorb, vital24, lab24)

print("Saving final dataset")
final.to_csv(OUTPUT_FILE, index=False)
print(f"Done! Final shape: {final.shape}")

Extracting 24-hour labs (efficient)...
Merging all features into final dataset...
Saving final dataset...
Done! Final shape: (62722, 101)


In [ ]:
print("ICU stays present in merged:", merged["ICUSTAY_ID"].notna().sum())
print("Unique ICU stays:", merged["ICUSTAY_ID"].nunique())

ICU stays present in merged: 0
Unique ICU stays: 0


In [ ]:
print(icustays.columns)
print(icustays.head())
print(icustays["HADM_ID"].isna().mean())
print(icustays["HADM_ID"].nunique())
print(admissions["HADM_ID"].nunique())

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE',
       'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID',
       'INTIME', 'OUTTIME', 'LOS'],
      dtype='object')
   ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID DBSOURCE FIRST_CAREUNIT  \
0     365         268   110404      280836  carevue           MICU   
1     366         269   106296      206613  carevue           MICU   
2     367         270   188028      220345  carevue            CCU   
3     368         271   173727      249196  carevue           MICU   
4     369         272   164716      210407  carevue            CCU   

  LAST_CAREUNIT  FIRST_WARDID  LAST_WARDID              INTIME  \
0          MICU            52           52 2198-02-14 23:27:38   
1          MICU            52           52 2170-11-05 11:05:29   
2           CCU            57           57 2128-06-24 15:05:20   
3          SICU            52           23 2120-08-07 23:12:42   
4           CCU            57           57 2186-12

In [ ]:
print(admissions["ADMITTIME"].dt.year.min(), admissions["ADMITTIME"].dt.year.max())

2100 2210


In [ ]:
print(admissions["ADMITTIME"].dt.year.unique()[:10])
print(icustays["INTIME"].dt.year.unique()[:10])

[2196 2153 2157 2139 2160 2126 2191 2177 2172 2108]
[2198 2170 2128 2120 2186 2141 2114 2147 2132 2164]


In [ ]:
print(admissions["HADM_ID"].dtype)
print(icustays["HADM_ID"].dtype)
print(admissions["HADM_ID"].head())
print(icustays["HADM_ID"].head())

int64
int64
0    165315
1    152223
2    124321
3    161859
4    129635
Name: HADM_ID, dtype: int64
0    110404
1    106296
2    188028
3    173727
4    164716
Name: HADM_ID, dtype: int64


In [ ]:
overlap = len(set(admissions["HADM_ID"]).intersection(icustays["HADM_ID"]))
print("Overlapping HADM_IDs:", overlap)
print("Total admissions:", len(admissions))
print("Total icu stays:", len(icustays))

Overlapping HADM_IDs: 57786
Total admissions: 58976
Total icu stays: 61532


In [ ]:
print("Admissions before cleaning:", admissions.shape)
adm_pat = admissions.copy()
adm_pat["ADMITTIME"] = pd.to_datetime(adm_pat["ADMITTIME"], errors="coerce")
adm_pat["DISCHTIME"] = pd.to_datetime(adm_pat["DISCHTIME"], errors="coerce")
print("Admissions after timestamp drop:", adm_pat.dropna(subset=["ADMITTIME","DISCHTIME"]).shape)

Admissions before cleaning: (58976, 19)
Admissions after timestamp drop: (58976, 19)


In [ ]:
test_merge = adm_pat.merge(icustays, on=["SUBJECT_ID","HADM_ID"], how="left")
print("ICU stays in test_merge:", test_merge["ICUSTAY_ID"].notna().sum())
print("Unique ICU stays in test_merge:", test_merge["ICUSTAY_ID"].nunique())

ICU stays in test_merge: 61532
Unique ICU stays in test_merge: 61532


In [ ]:
# Step A
sA = adm_pat.merge(icustays, on=["SUBJECT_ID","HADM_ID"], how="left")
print("Step A ICU stays:", sA["ICUSTAY_ID"].notna().sum())

Step A ICU stays: 61532


In [ ]:
adm_pat2 = adm_pat.copy()
adm_pat2["ADMITTIME"] = pd.to_datetime(adm_pat2["ADMITTIME"], errors="coerce")
adm_pat2["DISCHTIME"] = pd.to_datetime(adm_pat2["DISCHTIME"], errors="coerce")

sB = adm_pat2.merge(icustays, on=["SUBJECT_ID","HADM_ID"], how="left")
print("Step B ICU stays:", sB["ICUSTAY_ID"].notna().sum())

Step B ICU stays: 61532


In [ ]:
adm_pat3 = adm_pat2.dropna(subset=["ADMITTIME","DISCHTIME"])

sC = adm_pat3.merge(icustays, on=["SUBJECT_ID","HADM_ID"], how="left")
print("Step C ICU stays:", sC["ICUSTAY_ID"].notna().sum())

Step C ICU stays: 61532


In [ ]:
adm_pat4 = adm_pat3[
    adm_pat3["ADMITTIME"].dt.year.between(2000, 3000)
]

sD = adm_pat4.merge(icustays, on=["SUBJECT_ID","HADM_ID"], how="left")
print("Step D ICU stays:", sD["ICUSTAY_ID"].notna().sum())

Step D ICU stays: 61532


In [ ]:
print("ICU after merging:", merged["ICUSTAY_ID"].notna().sum())

ICU after merging: 0


In [76]:
df = pd.read_csv('clean_dataset.csv')

df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,mean_51222,mean_51265,mean_51300,min_50912,min_50971,min_50983,min_51006,min_51222,min_51265,min_51300
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,8.250,153.50,6.25,0.9,4.0,138.0,18.0,0.0,5.0,6.25
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,9.025,255.25,6.25,2.4,3.9,138.0,41.0,7.8,190.0,6.25
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,...,10.600,201.00,6.25,0.5,3.3,141.0,10.0,10.6,201.0,6.25
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,14.900,309.00,6.25,0.9,4.0,138.0,18.0,14.9,309.0,6.25
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,9.500,322.50,6.25,10.0,4.8,134.0,62.0,9.2,315.0,6.25
